In [81]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import eli5

import regex as re
import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier

from nltk.corpus import stopwords

import pickle


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [130]:
# df_reddit = pickle.load(open('../DataSet/df_reddit_for_model.pkl', 'rb'))
df_reddit = pickle.load(open('../DataSet/df_reddit_for_model.pkl', 'rb'))
EDA_df = pickle.load(open('../DataSet/EDA_all_for_model.pkl', 'rb'))
# df_reddit['target'] = df_reddit['subreddit'].replace({"NASA": 1, "Space_discussion": 0})
df_reddit.head(2)

,ent,text_merged,word_count*,sentiment_score*,target,polarity_score,polarity,polarity_VSA,lower_sent,tagged_sent,text_complexity
0,1,star shine saturn ring,4,0.0,1,0.0,1,1,star shine saturn ring,"[(star, NN), (shine, NN), (saturn, NN), (ring,...",92.80
1,2,smithsonian nation air space museum,5,0.0,1,0.0,1,1,smithsonian nation air space museum,"[(smithsonian, JJ), (nation, NN), (air, NN), (...",15.64


In [83]:
EDA_df.head(2)

,target,'',CC,CD,DT,EX,FW,IN,JJ,JJR,...,WP,WP$,WRB,ent,word_count*,sentiment_score*,polarity_score,polarity,polarity_VSA,text_complexity
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,4,0.0,0.0,1,1,92.80
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2,5,0.0,0.0,1,1,15.64


Defining X and Y variables and use train test split function

In [84]:
X = df_reddit['text_merged']
y = df_reddit['target']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    stratify=y,
                                                    random_state = 101)

In [85]:
cvec = CountVectorizer(min_df=4, max_df=1.0,
                       ngram_range=(1,2),max_features = 3000)

In [133]:
term_mat = cvec.fit_transform(df_reddit['text_merged'])

In [134]:
term_df = pd.DataFrame(term_mat.toarray(), 
                       columns=cvec.get_feature_names())
term_df.insert(0, 'targets', df_reddit['target'])

In [135]:
term_df.head()

,targets,aa,ab,abil,abl,abl get,abl see,aboard,aboard intern,aboard space,...,york,youlikebet,young,youtu,zealand,zero,zero graviti,zone,zoom,zubrin
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


First lets try Logistic Regression algorithm.

In [89]:
# !pip install pactools
# !/Users/amin/anaconda3/bin/python -m pip install pactools

### A comperhensive grid search on lr model Done.

pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('logreg', LogisticRegression())
])


pipe_params = {
    'cvec__max_features': [2_000, 4_000],
    'cvec__ngram_range': [(1,1), (1,2)],
    'logreg__penalty': ['l2', 'none'],
    'logreg__solver': ['newton-cg', 'sag'],
    'logreg__max_iter': [100, 300]
}

lr = GridSearchCV(pipe, # what object are we optimizing?
                  param_grid=pipe_params, # what parameters values are we searching?
                  cv=5, 
                  scoring='accuracy', 
                  verbose=2) 

lr.fit(X_train, y_train);
print('lr.best_params_\n', lr.best_params_)



### Here is the best results of above grid search.

```python

print('lr.best_params_\n', lr.best_params_)

lr.best_params_
 {'cvec__max_features': 4000, 'cvec__ngram_range': (1, 2), 
 'logreg__max_iter': 300, 'logreg__penalty': 'none', 'logreg__solver': 'sag'}
 
```

In [90]:
# lr.cv_results_

Going with best lr hyperparameters.

In [91]:
cvec = CountVectorizer(max_features= 4000,
                       min_df=4, max_df=1.0,
                       ngram_range=(1,2))
X_train_features = cvec.fit_transform(X_train)
X_test_features = cvec.transform(X_test)

lr = LogisticRegression(penalty='none', dual=False, tol=0.0001, 
                         C=1.0, fit_intercept=True, intercept_scaling=1, 
                         class_weight=None, random_state=None, solver='sag', 
                         max_iter=300, multi_class='auto', verbose=0, 
                         warm_start=False, n_jobs=None, l1_ratio=None)

lr.fit(X_train_features, y_train);

lr.score(X_train_features, y_train); 
print('train score', lr.score(X_train_features, y_train))

lr.score(X_test_features, y_test); 
print('test score', lr.score(X_test_features, y_test))

train score 0.7956666666666666
test score 0.7443333333333333


Checking out the Accuracy, Precision, Recall parameters.

In [93]:
preds = lr.predict(X_test_features)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# View confusion matrix

# plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

(0.7443333333333333, 0.768892149669846, 0.6986666666666667)

Applying gradient boosting algorithm.

### Using All CPUs

In [26]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=100, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=0, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
gboost_params = {
    'max_depth': [2,3,4],
    'n_estimators': [100, 125, 150],
    'learning_rate': [.08, .1, .12]
}
gs = GridSearchCV(gboost, param_grid=gboost_params, 
                  cv=3, verbose=1, n_jobs=-1)
gs.fit(X_train_features, y_train)

print(gs.best_score_)
gs.best_params_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    7.8s finished


0.7342222222222222


{'learning_rate': 0.12, 'max_depth': 4, 'n_estimators': 150}

just took ~ 8 seconds.
Now using 1 CPU.
### Using One CPU

In [27]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=100, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=0, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
gboost_params = {
    'max_depth': [2,3,4],
    'n_estimators': [100, 125, 150],
    'learning_rate': [.08, .1, .12]
}
gs = GridSearchCV(gboost, param_grid=gboost_params, 
                  cv=3, verbose=1, n_jobs=None)
gs.fit(X_train_features, y_train)

print(gs.best_score_)
gs.best_params_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:  1.0min finished


0.7330000000000001


{'learning_rate': 0.12, 'max_depth': 4, 'n_estimators': 150}

60 seconds compare to 8 seconds! 

Going with the best hyperparameters for gboost. Notice grid search convergence to max values.

In [29]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.12, 
                                    n_estimators=150, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=4, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=0, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
gboost.fit(X_train_features, y_train)
print(gboost.score(X_test_features, y_test))

0.7323333333333333


Checking out the Accuracy, Precision, Recall parameters.

In [30]:
preds = gboost.predict(X_test_features)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

(0.7323333333333333, 0.8065083553210203, 0.6113333333333333)

Using ANN model as follow.

### Using All CPUs.

In [31]:
mlp = MLPClassifier(hidden_layer_sizes=50, activation='relu', solver='adam', 
                    alpha=0.0001, batch_size='auto', learning_rate='constant', 
                    learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                    random_state=None, tol=0.0001, verbose=False, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
                    n_iter_no_change=10, max_fun=15000)


mlp_params = {
    'hidden_layer_sizes': [20,50],
    'activation': ['tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam']
}
mlp = GridSearchCV(mlp, param_grid=mlp_params, cv=3, verbose=1, n_jobs=-1)
mlp.fit(X_train_features, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  4.6min finished


NameError: name 'mlpg' is not defined

In [32]:
print(mlp.best_score_)
mlp.best_params_

0.7447777777777778


{'activation': 'tanh', 'hidden_layer_sizes': 20, 'solver': 'sgd'}

Going with the best hyperparameters for ANN. 

In [34]:
preds = mlp.predict(X_test_features)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

(0.7556666666666667, 0.7672473867595819, 0.734)

Checking out the Accuracy, Precision, Recall parameters.

In [23]:
mlp = MLPClassifier(hidden_layer_sizes=50, activation='relu', solver='sgd', 
                    alpha=0.0001, batch_size='auto', learning_rate='constant', 
                    learning_rate_init=0.001, power_t=0.5, max_iter=100, shuffle=True, 
                    random_state=None, tol=0.0001, verbose=False, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
                    n_iter_no_change=10, max_fun=15000)


mlp.fit(X_train_features, y_train)
print(mlp.score(X_test_features, y_test))

0.7406666666666667


## Lets use EDA data

In [35]:
EDA_df.head(2)

,target,'',CC,CD,DT,EX,FW,IN,JJ,JJR,...,WP,WP$,WRB,ent,word_count*,sentiment_score*,polarity_score,polarity,polarity_VSA,text_complexity
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,4,0.0,0.0,1,1,92.80
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2,5,0.0,0.0,1,1,15.64


Prepare the data frame as numpy.

In [46]:
y = EDA_df['target']
X = EDA_df.drop(columns = ['target'])
X_arr = np.array(X)
y_arr = np.array(y)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_arr,
                                                    y_arr,
                                                    test_size = 0.25,
                                                    stratify=y,
                                                    random_state = 101)

#### A comperhensive grid search on lr model.

In [64]:
# lr_model = LogisticRegression(penalty='none', dual=False, tol=0.0001, 
#                               C=1.0, fit_intercept=True, intercept_scaling=1, 
#                               class_weight=None, random_state=None, solver='sag', 
#                               max_iter=300, multi_class='auto', verbose=0, 
#                               warm_start=False, n_jobs=None, l1_ratio=None)

# pipe_params = {'lr_model__penalty': ['l2', 'none'], 
#                'lr_model__solver': ['newton-cg', 'sag'], 
#                'lr_model__max_iter': [100, 300]}


# lr = GridSearchCV(lr_model, param_grid=pipe_params, 
#                   cv=3, verbose=1, n_jobs=-1)

# lr.fit(X_train, y_train)

In [65]:
lr = LogisticRegression(solver='sag', 
                        max_iter=5000)

lr.fit(X_train, y_train);

lr.score(X_train, y_train); 
print('train score', lr.score(X_train, y_train))

lr.score(X_test, y_test); 
print('test score', lr.score(X_test, y_test))

train score 0.825
test score 0.8276666666666667


In [66]:
preds = lr.predict(X_test)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

(0.8276666666666667, 0.9685414680648237, 0.6773333333333333)

In [68]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, 
                                    n_estimators=100, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=0, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
gboost_params = {
    'max_depth': [2,3,4],
    'n_estimators': [100, 250, 500],
    'learning_rate': [.08, .1, .12]
}
gs = GridSearchCV(gboost, param_grid=gboost_params, 
                  cv=3, verbose=1, n_jobs=-1)
gs.fit(X_train, y_train)

print(gs.best_score_)
gs.best_params_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   21.1s finished


0.8345555555555556


{'learning_rate': 0.08, 'max_depth': 2, 'n_estimators': 100}

and going down with the best hyperparameters

In [69]:
gboost = GradientBoostingClassifier(loss='deviance', learning_rate=0.08, 
                                    n_estimators=100, subsample=1.0, criterion='friedman_mse', 
                                    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                    max_depth=2, min_impurity_decrease=0.0, min_impurity_split=None, 
                                    init=None, random_state=None, max_features=None, verbose=0, 
                                    max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, 
                                    n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
gboost.fit(X_train, y_train)
print(gboost.score(X_test, y_test))

0.8353333333333334


In [70]:
preds = gboost.predict(X_test)

# Save confusion matrix values
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# View confusion matrix

#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

(0.8353333333333334, 1.0, 0.6706666666666666)

### ANN using all CPUs

In [75]:
mlp = MLPClassifier(hidden_layer_sizes=50, activation='relu', solver='adam', 
                    alpha=0.0001, batch_size='auto', learning_rate='constant', 
                    learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                    random_state=None, tol=0.0001, verbose=False, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
                    n_iter_no_change=10, max_fun=15000)


mlp_params = {
    'hidden_layer_sizes': [20,50],
    'activation': ['tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam']
}
mlp = GridSearchCV(mlp, param_grid=mlp_params, cv=3, verbose=1, n_jobs=-1)
mlp.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    9.1s finished


GridSearchCV(cv=3, estimator=MLPClassifier(hidden_layer_sizes=50), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'hidden_layer_sizes': [20, 50],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [76]:
print(mlp.best_score_)
mlp.best_params_

0.8273333333333334


{'activation': 'relu', 'hidden_layer_sizes': 50, 'solver': 'adam'}

In [77]:
mlp = MLPClassifier(hidden_layer_sizes=50, activation='relu', solver='adam', 
                    alpha=0.0001, batch_size='auto', learning_rate='constant', 
                    learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, 
                    random_state=None, tol=0.0001, verbose=False, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, 
                    n_iter_no_change=10, max_fun=15000)


mlp.fit(X_train, y_train)
print(mlp.score(X_test, y_test))

0.8323333333333334


In [78]:
preds = mlp.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

# View confusion matrix
#plot_confusion_matrix(gs, X_test_features, y_test, cmap='Blues', values_format='d');
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Precision = tp / (tp + fp)
Recall = tp / (tp + fn)

Accuracy, Precision, Recall

(0.8323333333333334, 0.9911330049261083, 0.6706666666666666)